# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 100
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten

### Simulations Eingangsdaten

In [2]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus Polling - FIFO
- der Algorithmus weist den BEVs jeweils ein Zustand zu, die Zustände sind wartend (BEV parkt und will laden), ladend (BEV wird mit Solarenergie geladen), nicht parkend (BEV befindet sich nicht im Lademanagmentsystem)
- pro BEV Ladeleistung von z.B. 2kW (ladeleistung_pro_bev), wenn verfügbarer Solarstrom > Ladeleistung, dann wird der Solarstrom auf mehrere BEVs verteilt
- aktuell wartende Verbraucher werden in einer Warteliste (Pollingliste) abgearbeitet, nach einem variablen Zeitintervall (maximale_ladezeit, z.B. 30min) wird Verbraucher wieder hinten an die Warteliste angefügt. 
- dabei wird immer das erste BEV der Warteliste auf die Ladeliste genommen, unabhängig von der Länge der Parkzeit
- Wenn im Tagesverlauf die Anzahl bereits ladender BEVs größer wird als die Anzahl möglich ladender BEVs, wird das erste BEV der Ladeliste wieder nach hinten an die Warteliste gesetzt

### Verteilungsalgorithmus Polling - mit gleichmäßiger Verteilung
- wie FIFO Abarbeitung der BEVs auf der Warteliste
- allerdings werden BEVs nach dem sie einmal aufgeladen wurden auf eine neue Warteliste gesetzt mit geringerer Priorität => so soll gewährleistet werden, dass erstmal alle BEVs einmal aufgeladen werden, bevor BEVs mehrmals geladen werden 

### Prognose (Berücksichtigung der Parkdauer - Optimale Ausnutzung)
- Ziel ist das möglichst alle gleich viel bekommen und der Solarstrom optimal ausgenutzt wird ⇒ nichts geht verloren
- Berechnung des Ladeintervalls:
    - Wann sind während des angegebenen Parkintervalls am wenigsten andere BEVs am Warten? (Minimum Anzahl wartender BEVs in Parkintevall)
    - Ladestart: sobald Minimum Anzahl wartender BEVs in Parkintevall erreicht wird
    - Ladeende: nach fester Zeit oder nachdem Anzahl wartender BEVs nicht mehr Minimum Anzahl wartender BEVs entspricht
- Solarleistung wird zu jedem Zeitpunkt gleichmäßig unter ladenden BEVs verteilt, also Ladeleistung = Verfügbare Solarleistung / Ladende BEVs

### Prognose (Berücksichtigung der Parkdauer - Belohnung)
- Ziel: länger parkende BEVs sollen belohnt werden und mehr Solarenergie garantiert bekommen
- Möglichkeiten: mehr Solarenergie oder längeres Ladeintervall

In [3]:
import distributionAlgorithmPolling
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecast

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()

# Auswahl des Algorithmus
# algorithm = distributionAlgorithmPolling.Algorithm.POLLING_FIFO
algorithm = distributionAlgorithmPolling.Algorithm.POLLING_EVEN_DISTRIBUTION
distributionAlgorithmPolling.start_simulation(solarpeakleistung, maximale_ladezeit, ladeleistung_pro_bev, simulation_day, bev_data, table_dict, simulation_data, algorithm)

#distributionAlgorithmForecast.start_algorithm(simulation_data, simulation_day, maximale_ladezeit)

[1, 5, 15, 23, 37, 43, 47, 51, 53, 57, 61, 71, 75, 76, 81, 85, 86] 0 Waiting list with Id removed
[5, 15, 23, 37, 43, 47, 51, 53, 57, 61, 71, 75, 76, 81, 85, 86, 4, 8, 10, 11, 14, 17, 20, 24, 27, 28, 29, 32, 33, 34, 39, 41, 46, 48, 49, 54, 55, 56, 60, 62, 63, 65, 68, 72, 74, 78, 80, 84, 92, 94, 98] 1 Waiting list with Id removed
[15, 23, 37, 43, 47, 51, 53, 57, 61, 71, 75, 76, 81, 85, 86, 4, 8, 10, 11, 14, 17, 20, 24, 27, 28, 29, 32, 33, 34, 39, 41, 46, 48, 49, 54, 55, 56, 60, 62, 63, 65, 68, 72, 74, 78, 80, 84, 92, 94, 98] 5 Waiting list with Id removed
[23, 37, 43, 47, 51, 53, 57, 61, 71, 75, 76, 81, 85, 86, 4, 8, 10, 11, 14, 17, 20, 24, 27, 28, 29, 32, 33, 34, 39, 41, 46, 48, 49, 54, 55, 56, 60, 62, 63, 65, 68, 72, 74, 78, 80, 84, 92, 94, 98, 2, 12, 16, 19, 45, 50, 52, 58, 67, 87, 89, 90, 95, 99] 15 Waiting list with Id removed
[37, 43, 47, 51, 53, 57, 61, 71, 75, 76, 81, 85, 86, 4, 8, 10, 11, 14, 17, 20, 24, 27, 28, 29, 32, 33, 34, 39, 41, 46, 48, 49, 54, 55, 56, 60, 62, 63, 65, 68

[41, 46, 48, 49, 54, 55, 56, 62, 63, 65, 68, 72, 74, 78, 80, 84, 94, 98, 2, 12, 16, 19, 45, 50, 52, 58, 67, 87, 89, 90, 95, 99, 18, 42, 88, 97, 22, 82, 9, 40, 96] 39 Waiting list with Id removed
interrupted charging process
[41, 46, 48, 49, 54, 55, 56, 62, 63, 65, 68, 72, 74, 78, 80, 84, 94, 98, 2, 12, 16, 45, 50, 52, 58, 67, 87, 89, 90, 95, 99, 18, 42, 88, 97, 22, 82, 9, 40, 96, 25, 30, 70, 28] 19 Waiting list with Id removed
[46, 48, 49, 54, 55, 56, 62, 63, 65, 68, 72, 74, 78, 80, 84, 94, 98, 2, 12, 16, 45, 50, 52, 58, 67, 87, 89, 90, 95, 99, 18, 42, 88, 97, 22, 82, 9, 40, 96, 25, 30, 70, 28] 41 Waiting list with Id removed
[48, 49, 54, 55, 56, 62, 63, 65, 68, 72, 74, 78, 80, 84, 94, 98, 2, 12, 16, 45, 50, 52, 58, 67, 87, 89, 90, 95, 99, 18, 42, 88, 97, 22, 82, 9, 40, 96, 25, 30, 70, 28] 46 Waiting list with Id removed
[48, 49, 54, 55, 56, 62, 63, 65, 68, 72, 74, 78, 80, 84, 94, 98, 2, 12, 16, 45, 50, 52, 58, 67, 87, 89, 95, 99, 18, 42, 88, 97, 22, 82, 9, 40, 96, 25, 30, 70, 28] 90 W

[96, 30, 70, 26, 31, 35, 36, 38, 66, 73, 77, 3, 6, 7, 21, 44, 59, 69, 79, 83, 91, 93, 64, 13, 89] 40 Waiting list with Id removed
[30, 70, 26, 31, 35, 36, 38, 66, 73, 77, 3, 6, 7, 21, 44, 59, 69, 79, 83, 91, 93, 64, 13, 89] 96 Waiting list with Id removed
[30, 26, 31, 35, 36, 38, 66, 73, 77, 3, 6, 7, 21, 44, 59, 69, 79, 83, 91, 93, 64, 13, 89] 70 Waiting list with Id removed


### Ergebnis Algorithmus

In [4]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag)

>**Ergebnis**<br>Geladene BEVs: 60 von 100<br>Unterbrochene Aufladevorgänge 5<br>Aufgeladene Solarenergie insgesamt: 275.22 kWh<br>Ungenutzte Solarenergie insgesamt: 0 kWh

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [5]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict)

interactive(children=(IntSlider(value=480, description='Minute: ', max=960, min=480), Output()), _dom_classes=…

In [6]:
import figureGeneration

figureGeneration.create_charging_power_figure(simulation_day)

In [7]:
figureGeneration.create_bev_number_figure(simulation_data)